In [1]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [2]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [21]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

vectorizer = CountVectorizer()

finallist=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,createfile):
    tokenize_words=[]
    maintext=[]
    listofimpcandidate=[]
    listwithfinalsentence=[]
    file1 = open(createfile, "w", encoding="utf-8")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    titleno = soup.find_all('title')
    text=soup.find_all('text')
    instances = {}
    my_id = 0
    for n,tit,maintitle in zip(docno,text,titleno):
        tit=tit.get_text()
        tit=str(tit)
        sentences=sentence_tokenize.sentence_split(tit, lang='gu')
        for t in sentences:
            sen=removestopwords(t)
            #sen=t
            #print("this is sen",sen)
            maintext.append(sen)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in maintext]
        list3 = [s for s in list3 if s]
        
        bag_of_words = vectorizer.fit_transform(list3)
        #This process encodes our original data into topic encoded data
        lda = LatentDirichletAllocation(n_components = 2)
        #svd = TruncatedSVD(n_components = 2)
        lsa = lda.fit_transform(bag_of_words)
        topic_encoded_df = pd.DataFrame(lsa, columns=["topic1", "topic2"])
        topic_encoded_df["doc"]= list3
        avg1=topic_encoded_df["topic1"].mean()
        avg2=topic_encoded_df["topic2"].mean()
        topic_encoded_df['topic1'] = np.where(topic_encoded_df['topic1'] < avg1, 0, topic_encoded_df['topic1'])
        topic_encoded_df['topic2'] = np.where(topic_encoded_df['topic2'] < avg2, 0, topic_encoded_df['topic2'])
        sum_column = topic_encoded_df["topic1"] + topic_encoded_df["topic2"]
        topic_encoded_df["sum_column"]=sum_column
        sort_by_sumcolumn = topic_encoded_df.sort_values('sum_column',ascending=False)
        compressionratio = round((len(sort_by_sumcolumn.index)*30)/100)
        #print(compressionratio)
        final_df2=sort_by_sumcolumn['doc'].head(compressionratio)
        listtoretrieve=list(final_df2.index.values)
        listtoretrieve.sort()
        originalsen=[]
        for t in sentences:    
            originalsen.append(t)
        df4 = pd.DataFrame(originalsen,columns=['sentences'])
        #print(df4)
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=df4['sentences'].loc[i]
        #print(finaldraft)            
#         originalfull_str = convert_list_to_string(finaldraft)
#         originalfull_str=" ".join(originalfull_str.split())
#         n=str(n)
        file1.write(finaldraft)
        instances = {}    
        
    return instances

In [22]:
import os

#for root, dirs, files in os.walk("/home/harsh/Documents/gujdocs/gsf2003/"):
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization dataset/sports/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file(os.path.join(root, file),"G:/Movies/gujarati Text summarization_lda/sports/"+file)

In [23]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level

In [24]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [25]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]

class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file1(filename,reference):
    tokenize_words=[]
    tokenize_words1=[]
    
    #file1 = open(reference, "r")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #print(data)
    
    for tokenize in indic_tokenize.trivial_tokenize(data):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
    originalfull_str = convert_list_to_string(tokenize_words)
    senlateron=[]
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    i=1
    with codecs.open(reference+"a1_"+str(i)+".txt", encoding='utf-8') as f1:
        data1 = f1.read()
    i=i+1
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    senlateron1=[]
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-W: %f' % rouge_w)
    rougew.append(rouge_w)
    print(rouge1)
    average1 = Average(rouge1)
    print("average of rouge1",average1)
    print(rouge2)
    average2 = Average(rouge2)
    print("average of rouge2",average2)
    print(rougel)
    averagel = Average(rougel)
    print("average of rougel",averagel)
    print(rougew)
    averagew = Average(rougew)
    print("average of rougew",averagew)
   
    instances = {}


    return instances

In [26]:
import os
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization_lda/sports/"):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file1(os.path.join(root, file),"C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/sports/annotator1/")

[0.8568061284619917]
average of rouge1 0.8568061284619917
[0.6654867256637168]
average of rouge2 0.6654867256637168
[0.8568061284619917]
average of rougel 0.8568061284619917
[0.28065864380651956]
average of rougew 0.28065864380651956
[0.8568061284619917, 0.6814621409921671]
average of rouge1 0.7691341347270794
[0.6654867256637168, 0.37777777777777777]
average of rouge2 0.5216322517207472
[0.8568061284619917, 0.6814621409921671]
average of rougel 0.7691341347270794
[0.28065864380651956, 0.22210544801967314]
average of rougew 0.2513820459130963
[0.8568061284619917, 0.6814621409921671, 0.6292134831460674]
average of rouge1 0.7224939175334087
[0.6654867256637168, 0.37777777777777777, 0.382595648912228]
average of rouge2 0.47528671745124085
[0.8568061284619917, 0.6814621409921671, 0.6292134831460674]
average of rougel 0.7224939175334087
[0.28065864380651956, 0.22210544801967314, 0.18128746743842913]
average of rougew 0.22801718642154062
[0.8568061284619917, 0.6814621409921671, 0.62921348314

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781]
average of rouge1 0.6663977210617548
[0.6654867256637168, 0.37777777777777777, 0.382595648912228, 0.23058446757405926, 0.3944476576055523, 0.3693430656934307, 0.3504706734250543, 0.4561186650185414, 0.4453280318091451, 0.309328968903437, 0.42112578179291177, 0.29259896729776247, 0.4070494599204093, 0.40887693342299936]
average of rouge2 0.39365234462978754
[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7637203928365107, 0.6384839650145773, 0.636297903109183, 0.7765432098765432, 0.7531436135009928, 0.5147058823529411, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781]
average of rougel 0.6660277669307988
[0.28065864380651956, 0.22210544801967314, 0

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495]
average of rouge1 0.6477204361207678
[0.6654867256637168, 0.37777777777777777, 0.382595648912228, 0.23058446757405926, 0.3944476576055523, 0.3693430656934307, 0.3504706734250543, 0.4561186650185414, 0.4453280318091451, 0.309328968903437, 0.42112578179291177, 0.29259896729776247, 0.4070494599204093, 0.40887693342299936, 0.43124999999999997, 0.3546099290780142, 0.25597269624573377, 0.3204837490551776, 0.3769559032716927, 0.36166924265842354]
average of rouge2 0.3806037172563034
[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7637203928365107, 0.6384839650145773

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922]
average of rouge1 0.6343554141034591
[0.6654867256637168, 0.37777777777777777, 0.382595648912228, 0.23058446757405926, 0.3944476576055523, 0.3693430656934307, 0.3504706734250543, 0.4561186650185414, 0.4453280318091451, 0.309328968903437, 0.42112578179291177, 0.29259896729776247, 0.4070494599204093, 0.40887693342299936, 0.43124999999999997, 0.3546099290780142, 0.25597269624573377, 0.3204837490551776, 0.3769559032716927, 0.36166924265842354, 0.29566694987255737, 0.2766865926558497, 0.391432791728

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269]
average of rouge1 0.6316993822470904
[0.6654867256637168, 0.37777777777777777, 0.382595648912228, 0.23058446757405926, 0.3944476576055523, 0.3693430656934307, 0.3504706734250543, 0.4561186650185414, 0.4453280318091451, 0.309328968903437, 0.42112578179291177, 0.29259896729776247, 0.4070494599204093, 0.40887693342299936, 0.43124999999999997, 0.3546099290780142, 0.25597269624573377, 0.3204837490551776, 0.37695590327169

...

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269, 0.7251217814892136, 0.7412868632707774, 0.7217332430602572, 0.49166666666666664, 0.708825573314802, 0.6949602122015914, 0.6588579795021963, 0.5402029664324747, 0.39253996447602135, 0.614920874152223, 0.7624309392265194, 0.7268041237113401, 0.7053072625698323, 0.6628489620615605, 0.6473988439306358, 0.5548686244204019, 0.6394940267041461, 0.4080914687774846, 0.5745682888540031, 0.7367750159337156, 0.6942266571632217,

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269, 0.7251217814892136, 0.7412868632707774, 0.7217332430602572, 0.49166666666666664, 0.708825573314802, 0.6949602122015914, 0.6588579795021963, 0.5402029664324747, 0.39253996447602135, 0.614920874152223, 0.7624309392265194, 0.7268041237113401, 0.7053072625698323, 0.6628489620615605, 0.6473988439306358, 0.5548686244204019, 0.6394940267041461, 0.4080914687774846, 0.5745682888540031, 0.7367750159337156, 0.6942266571632217,

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269, 0.7251217814892136, 0.7412868632707774, 0.7217332430602572, 0.49166666666666664, 0.708825573314802, 0.6949602122015914, 0.6588579795021963, 0.5402029664324747, 0.39253996447602135, 0.614920874152223, 0.7624309392265194, 0.7268041237113401, 0.7053072625698323, 0.6628489620615605, 0.6473988439306358, 0.5548686244204019, 0.6394940267041461, 0.4080914687774846, 0.5745682888540031, 0.7367750159337156, 0.6942266571632217,

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269, 0.7251217814892136, 0.7412868632707774, 0.7217332430602572, 0.49166666666666664, 0.708825573314802, 0.6949602122015914, 0.6588579795021963, 0.5402029664324747, 0.39253996447602135, 0.614920874152223, 0.7624309392265194, 0.7268041237113401, 0.7053072625698323, 0.6628489620615605, 0.6473988439306358, 0.5548686244204019, 0.6394940267041461, 0.4080914687774846, 0.5745682888540031, 0.7367750159337156, 0.6942266571632217,

[0.8568061284619917, 0.6814621409921671, 0.6292134831460674, 0.4812150279776179, 0.7660311958405546, 0.6384839650145773, 0.636297903109183, 0.7777777777777779, 0.7531436135009928, 0.5163398692810457, 0.6856349757113117, 0.4329896907216495, 0.7461669505962522, 0.7280053727333781, 0.7578027465667916, 0.5680566483084185, 0.4531516183986371, 0.5630188679245283, 0.6917613636363636, 0.5910493827160495, 0.45801526717557245, 0.4586530264279625, 0.6415929203539823, 0.7029569892473118, 0.6432584269662922, 0.6609808102345416, 0.4490145672664953, 0.6302395209580838, 0.7201618341200269, 0.7251217814892136, 0.7412868632707774, 0.7217332430602572, 0.49166666666666664, 0.708825573314802, 0.6949602122015914, 0.6588579795021963, 0.5402029664324747, 0.39253996447602135, 0.614920874152223, 0.7624309392265194, 0.7268041237113401, 0.7053072625698323, 0.6628489620615605, 0.6473988439306358, 0.5548686244204019, 0.6394940267041461, 0.4080914687774846, 0.5745682888540031, 0.7367750159337156, 0.6942266571632217,